In [3]:
import pandas as pd
import numpy as np
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings

warnings.simplefilter("ignore", category=UserWarning)

In [4]:
# Data Preprocessing
data = pd.read_csv('Diabetes.csv')
data['Sex'] = data['Sex'].map({'female': 0, 'male': 1})
label_encoder = LabelEncoder()
data['Prediction'] = label_encoder.fit_transform(data['Prediction'])

# Feature Selection
feature_columns = data.columns.difference(['ID No', 'Name', 'Prediction']).tolist()

In [5]:
# Initialize tracking for results
accuracy_results = []
column_importance = {col: 0 for col in feature_columns}


In [23]:
def pad_features(x, n_qubits):
    """
    Pad the feature vector to the nearest power of 2 for compatibility with quantum circuits.
    """
    target_length = 2**n_qubits  # Calculate the target length
    padding_length = target_length - len(x)  # Determine the amount of padding needed
    if padding_length < 0:
        raise ValueError("Feature vector length exceeds the required size for embedding.")
    return np.pad(x, (0, padding_length), 'constant')

In [36]:
def evaluate_model(selected_columns):
    """
    Train and evaluate a quantum model on the selected features.
    """
    X = data[selected_columns].values
    y = data['Prediction'].values

    y_encoded = (y == 2).astype(int)  # Adjust encoding for binary classification

    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Assuming this is where you're calculating the number of qubits
    n_qubits = int(np.ceil(np.log2(len(selected_columns))))  # Calculate required qubits

    # Check for valid number of qubits
    if n_qubits <= 0:
        n_qubits = 1
    dev = qml.device('default.qubit', wires=n_qubits)

    params = np.random.normal(0, np.pi, size=n_qubits * 2)


    # Quantum circuit with parameterized gates
    @qml.qnode(dev)
    def quantum_circuit(params, x):
        # Apply an RX gate to the first qubit
        qml.RX(params[0], wires=0)

        # Apply CNOT gates
        for i in range(n_qubits - 1):
            qml.CRX(params[n_qubits + i], wires=[i, i + 1])

        # Only apply the controlled gate if there is more than 1 qubit
        if n_qubits > 1:
            qml.CRX(params[-1], wires=[n_qubits - 1, 0])  # Apply controlled gate
        else:
            # Apply a simple RX gate if only 1 qubit exists
            qml.RX(params[-1], wires=0)

        return qml.expval(qml.PauliZ(0))

    # Predict on the training set
    y_pred = np.array([quantum_circuit(params, X_train[i]) for i in range(len(X_train))])
    y_pred = (y_pred > 0).astype(int)

    # Calculate accuracy
    accuracy = np.mean(y_pred == y_train)
    return accuracy


In [37]:
# Evaluate Model
max_columns = len(feature_columns)
iterations = 5
sample = 1

In [38]:
for num_columns in range(1, max_columns + 1):
    for _ in range(iterations):
        if sample < 41:
            selected_columns = np.random.choice(feature_columns, size=num_columns, replace=False)

            accuracy = evaluate_model(selected_columns)

            if accuracy > 0.5 and len(selected_columns) > 2:
                print(f"Sample {sample} - Circuit 3")
                print(f"\tNumber of columns: {len(selected_columns)}")
                print(f"\tAccuracy: {accuracy:.2%}")
                accuracy_results.append((selected_columns, accuracy))
                sample += 1

Sample 1 - Circuit 3
	Number of columns: 3
	Accuracy: 65.83%
Sample 2 - Circuit 3
	Number of columns: 4
	Accuracy: 65.83%
Sample 3 - Circuit 3
	Number of columns: 5
	Accuracy: 65.83%
Sample 4 - Circuit 3
	Number of columns: 6
	Accuracy: 65.83%
Sample 5 - Circuit 3
	Number of columns: 6
	Accuracy: 65.83%
Sample 6 - Circuit 3
	Number of columns: 7
	Accuracy: 65.83%
Sample 7 - Circuit 3
	Number of columns: 7
	Accuracy: 65.83%
Sample 8 - Circuit 3
	Number of columns: 8
	Accuracy: 65.83%
Sample 9 - Circuit 3
	Number of columns: 8
	Accuracy: 65.83%
Sample 10 - Circuit 3
	Number of columns: 8
	Accuracy: 65.83%
Sample 11 - Circuit 3
	Number of columns: 9
	Accuracy: 65.83%
Sample 12 - Circuit 3
	Number of columns: 10
	Accuracy: 65.83%
Sample 13 - Circuit 3
	Number of columns: 10
	Accuracy: 65.83%
Sample 14 - Circuit 3
	Number of columns: 11
	Accuracy: 65.83%
Sample 15 - Circuit 3
	Number of columns: 11
	Accuracy: 65.83%
Sample 16 - Circuit 3
	Number of columns: 12
	Accuracy: 65.83%
Sample 17 - 

## Trying out new circuit type 

In [39]:
import pandas as pd
import numpy as np
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings

In [40]:
warnings.simplefilter("ignore", category=UserWarning)

In [41]:
# Data Preprocessing
data = pd.read_csv('Diabetes.csv')
data['Sex'] = data['Sex'].map({'female': 0, 'male': 1})
label_encoder = LabelEncoder()
data['Prediction'] = label_encoder.fit_transform(data['Prediction'])

In [42]:
# Feature Selection
feature_columns = data.columns.difference(['ID No', 'Name', 'Prediction']).tolist()

# Initialize tracking for results
accuracy_results = []
column_importance = {col: 0 for col in feature_columns}

In [43]:
def pad_features(x, n_qubits):
    """
    Pad the feature vector to the nearest power of 2 for compatibility with quantum circuits.
    """
    target_length = 2**n_qubits  # Calculate the target length
    padding_length = target_length - len(x)  # Determine the amount of padding needed
    if padding_length < 0:
        raise ValueError("Feature vector length exceeds the required size for embedding.")
    return np.pad(x, (0, padding_length), 'constant')


In [51]:
def evaluate_model(selected_columns):
    """
    Train and evaluate a quantum model on the selected features.
    """
    X = data[selected_columns].values
    y = data['Prediction'].values

    y_encoded = (y == 2).astype(int)  # Adjust encoding for binary classification

    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Assuming this is where you're calculating the number of qubits
    n_qubits = int(np.ceil(np.log2(len(selected_columns))))  # Calculate required qubits

    # Check for valid number of qubits
    if n_qubits <= 0:
        n_qubits = 1
    dev = qml.device('default.qubit', wires=n_qubits)

    params = np.random.normal(0, np.pi, size=n_qubits * 3)  # Increased parameters for multiple layers

    # Quantum circuit with parameterized gates
    @qml.qnode(dev)
    def quantum_circuit(params, x):
        # Feature map: Use multiple RX and RY gates to encode the features
        for i in range(n_qubits):
            qml.RX(x[i % len(x)], wires=i)  # Encode feature into RX gate

        # Layered Quantum Circuit (Deep Circuit)
        for layer in range(2):  # 2 layers for example
            for i in range(n_qubits - 1):
                qml.CNOT(wires=[i, i + 1])  # Entangling qubits
                qml.RY(params[i % len(params)], wires=i)  # Rotation to encode data

            # Apply controlled gates in each layer
            if n_qubits > 1:
                qml.CRX(params[-1], wires=[n_qubits - 1, 0])  # Cross entanglement between qubits

        # Measurement: We can also consider using different Pauli operators to test different behaviors
        return qml.expval(qml.PauliZ(0))

    # Predict on the training set
    y_pred = np.array([quantum_circuit(params, X_train[i]) for i in range(len(X_train))])
    y_pred = (y_pred > 0).astype(int)

    # Calculate accuracy
    accuracy = np.mean(y_pred == y_train)
    return accuracy

In [52]:
# Evaluate Model
max_columns = len(feature_columns)
iterations = 5
sample = 1

In [54]:
for num_columns in range(1, max_columns + 1):
    for _ in range(iterations):
        if sample < 41:
            selected_columns = np.random.choice(feature_columns, size=num_columns, replace=False)

            accuracy = evaluate_model(selected_columns)

            if accuracy > 0.6 and len(selected_columns) > 2:
                print(f"Sample {sample} - Quantum Circuit with Feature Map")
                print(f"\tNumber of columns: {len(selected_columns)}")
                print(f"Selected Columns: {selected_columns}")
                print(f"\tAccuracy: {accuracy:.2%}")
                accuracy_results.append((selected_columns, accuracy))
                sample += 1

Sample 5 - Quantum Circuit with Feature Map
	Number of columns: 4
Selected Columns: ['D.Bilirubin' 'Apolipoprotein-A1' 'SGPT' 'TGL']
	Accuracy: 62.50%
Sample 6 - Quantum Circuit with Feature Map
	Number of columns: 6
Selected Columns: ['Alk.Phosphate' 'UrineCreatinine' 'Chol/HDL.Ratio' 'MGV' 'Albumin' 'SGOT']
	Accuracy: 61.67%
Sample 7 - Quantum Circuit with Feature Map
	Number of columns: 6
Selected Columns: ['Post Prandial' 'Apolipoprotein-A1' 'TGL' 'D.Bilirubin' 'GTT 2 Hr' 'LPO']
	Accuracy: 65.83%
Sample 8 - Quantum Circuit with Feature Map
	Number of columns: 7
Selected Columns: ['T.Cholesterol' 'Insulin' 'UricAcid' 'TGL' 'SGPT' 'TSH3'
 'Serum Creatinine']
	Accuracy: 60.83%
Sample 9 - Quantum Circuit with Feature Map
	Number of columns: 7
Selected Columns: ['Chol/HDL.Ratio' 'TGL' 'GTT 1/2 Hr' 'GTT 1-1/2 Hr' 'hs-CRP'
 'Alk.Phosphate' 'LDL']
	Accuracy: 60.83%
Sample 10 - Quantum Circuit with Feature Map
	Number of columns: 7
Selected Columns: ['SGPT' 'SOD' 'MGV' 'Serum Creatinine' 'L